In [1]:
import pandas as pd
import numpy as np

## Data Processing

In this part, we processed several datasets and calculated the maximum, minimum and mode of Workload

In [ ]:
episodes = pd.read_excel('data/Episodes.xlsx')

In [67]:
episodes['AdmitDate'] = pd.to_datetime(episodes['AdmitDate'], format='%m/%d/%Y')
episodes['DischargeDate'] = pd.to_datetime(episodes['DischargeDate'], format='%Y-%m-%d')

episodes

,PID,EpisodeID,ContractType,AdmitDate,DischargeDate
0,530,1,CT1,2019-12-01,2020-01-30
1,1107,2,CT1,2019-12-01,NaT
2,1107,3,CT2,2019-12-01,2019-12-07
3,1107,4,CT2,2019-12-01,2019-12-02
4,1249,5,CT1,2019-12-01,NaT
...,...,...,...,...,...
40993,1771,56813,CT1,2019-01-01,2019-01-15
40994,1177,56814,CT1,2019-01-01,2019-01-14
40995,1187,56815,CT1,2019-01-01,2019-02-07
40996,1771,56816,CT1,2019-01-01,2019-01-18


In [68]:
longdf = pd.DataFrame(columns=['Date', 'PID', 'EpisodeID', 'ContractType', 'WorkLoadMode','WorkLoadMin','WorkLoadMax'])

for index, row in episodes.iterrows():
    PID = row['PID']
    EpisodeID = row['EpisodeID']
    ContractType = row['ContractType']
    AdmitDate = row['AdmitDate']
    DischargeDate = row['DischargeDate']
    
    if pd.isnull(DischargeDate):
        today = datetime.date.today()
        DischargeDate = today.strftime('%Y-%m-%d')
        
    date_list = [datetime.datetime.strftime(x,'%Y-%m-%d') for x in list(pd.date_range(start=AdmitDate, end=DischargeDate))]
            
    if ContractType == 'CT1':
        workloadmin =  [6.33 for i in range(min(3,len(date_list)))] + [2.86 for i in range(len(date_list)-3)]
        workloadmode = [10.33 for i in range(min(3,len(date_list)))] + [6.57 for i in range(len(date_list)-3)]
        workloadmax =  [22 for i in range(min(3,len(date_list)))] + [18.35 for i in range(len(date_list)-3)]
    elif ContractType == 'CT2':
        workloadmin =  [10.33 for i in range(min(3,len(date_list)))] + [4.71 for i in range(len(date_list)-3)]
        workloadmode = [27.67 for i in range(min(3,len(date_list)))] + [14.29 for i in range(len(date_list)-3)]
        workloadmax =  [50.33 for i in range(min(3,len(date_list)))] + [33.93 for i in range(len(date_list)-3)]
    
    
    episode = pd.DataFrame(columns=['Date', 'PID', 'EpisodeID', 'ContractType', 'WorkLoadMode','WorkLoadMin','WorkLoadMax'])
    episode['Date'] = date_list
    episode['PID'] = PID
    episode['EpisodeID'] = EpisodeID
    episode['ContractType'] = ContractType
    episode['WorkLoadMode'] = workloadmode
    episode['WorkLoadMin'] = workloadmin
    episode['WorkLoadMax'] = workloadmax
    longdf = longdf.append(episode,ignore_index=True)
    
    print("progress: {0}%".format(round((index + 1) * 100 / len(episodes), 2)), end="\r")

In [69]:
longdf

,Date,PID,EpisodeID,ContractType,WorkLoadMode,WorkLoadMin,WorkLoadMax
0,2019-12-01,530,1,CT1,10.33,6.33,22.00
1,2019-12-02,530,1,CT1,10.33,6.33,22.00
2,2019-12-03,530,1,CT1,10.33,6.33,22.00
3,2019-12-04,530,1,CT1,6.57,2.86,18.35
4,2019-12-05,530,1,CT1,6.57,2.86,18.35
...,...,...,...,...,...,...,...
1381253,2019-01-14,1807,56817,CT1,6.57,2.86,18.35
1381254,2019-01-15,1807,56817,CT1,6.57,2.86,18.35
1381255,2019-01-16,1807,56817,CT1,6.57,2.86,18.35
1381256,2019-01-17,1807,56817,CT1,6.57,2.86,18.35


In [70]:
longdf.to_csv('longdf.csv')

In [71]:
date_pid = pd.DataFrame()
date_pid['WorkLoadMode'] = longdf.groupby(['Date', 'PID'])['WorkLoadMode'].sum()
date_pid['WorkLoadMin'] = longdf.groupby(['Date', 'PID'])['WorkLoadMin'].sum()
date_pid['WorkLoadMax'] = longdf.groupby(['Date', 'PID'])['WorkLoadMax'].sum()
date_pid.reset_index(inplace=True)

In [72]:
date_pid

,Date,PID,WorkLoadMode,WorkLoadMin,WorkLoadMax
0,2019-01-01,6,304.37,113.63,553.63
1,2019-01-01,8,536.06,202.60,978.27
2,2019-01-01,9,155.32,75.30,304.99
3,2019-01-01,10,55.34,20.66,100.66
4,2019-01-01,11,110.68,41.32,201.32
...,...,...,...,...,...
382240,2021-04-14,716,6.57,2.86,18.35
382241,2021-04-15,716,6.57,2.86,18.35
382242,2021-04-16,716,6.57,2.86,18.35
382243,2021-04-17,716,6.57,2.86,18.35


In [73]:
date_pid.to_csv('date_pid.csv')

In [75]:
SNFs = pd.read_excel('data/SNFs.xlsx')

In [113]:
lat_l, lon_l = [], []
for index, row in SNFs.iterrows():
    ProviderPracticeAddress1 = row['ProviderPracticeAddress1']
    ProviderPracticeCity = row['ProviderPracticeCity']
    ProviderPracticeStateCode = row['ProviderPracticeStateCode']
    ProviderPracticeZipCode = row['ProviderPracticeZipCode']
    
    address = ProviderPracticeAddress1+', ' +ProviderPracticeCity+', '+ProviderPracticeStateCode+' '+str(ProviderPracticeZipCode)
    try:
        locator = Nominatim(user_agent='myGeocoder')
        location = locator.geocode(address)
        lat = location.latitude
        lon = location.longitude
    except:
        lat = np.nan
        lon = np.nan
    lat_l.append(lat)
    lon_l.append(lon)

    print("progress: {0}%".format(round((index + 1) * 100 / len(SNFs), 2)), end="\r")
    

In [114]:
SNFs['latitude'] = lat_l
SNFs['longitude'] = lon_l

In [138]:
PID_latlon = SNFs[['PID','latitude','longitude']]
PID_latlon

,PID,latitude,longitude
0,1,NaN,NaN
1,2,32.481913,-99.746158
2,3,NaN,NaN
3,4,NaN,NaN
4,5,NaN,NaN
...,...,...,...
1808,1809,31.785765,-96.452405
1809,1810,29.307465,-97.129599
1810,1811,32.968797,-102.835492
1811,1812,29.306790,-97.130264


In [124]:
PID_latlon[PID_latlon.isnull().T.any()]

,PID,latitude,longitude
0,1,NaN,NaN
2,3,NaN,NaN
3,4,NaN,NaN
4,5,NaN,NaN
12,13,NaN,NaN
...,...,...,...
1784,1785,NaN,NaN
1786,1787,NaN,NaN
1787,1788,NaN,NaN
1800,1801,NaN,NaN


In [119]:
result = date_pid.merge(PID_latlon, on='PID', how='left')

In [120]:
result

,Date,PID,WorkLoadMode,WorkLoadMin,WorkLoadMax,latitude,longitude
0,2019-01-01,6,304.37,113.63,553.63,30.583101,-96.275736
1,2019-01-01,8,536.06,202.60,978.27,33.040040,-96.832425
2,2019-01-01,9,155.32,75.30,304.99,32.887383,-96.765846
3,2019-01-01,10,55.34,20.66,100.66,33.853630,-98.536196
4,2019-01-01,11,110.68,41.32,201.32,32.875667,-96.613629
...,...,...,...,...,...,...,...
382240,2021-04-14,716,6.57,2.86,18.35,30.702646,-95.541119
382241,2021-04-15,716,6.57,2.86,18.35,30.702646,-95.541119
382242,2021-04-16,716,6.57,2.86,18.35,30.702646,-95.541119
382243,2021-04-17,716,6.57,2.86,18.35,30.702646,-95.541119


In [121]:
result[result.isnull().T.any()]

,Date,PID,WorkLoadMode,WorkLoadMin,WorkLoadMax,latitude,longitude
6,2019-01-01,13,110.68,41.32,201.32,NaN,NaN
8,2019-01-01,15,27.67,10.33,50.33,NaN,NaN
9,2019-01-01,17,318.02,128.62,591.30,NaN,NaN
12,2019-01-01,20,38.00,16.66,72.33,NaN,NaN
35,2019-01-01,67,27.67,10.33,50.33,NaN,NaN
...,...,...,...,...,...,...,...
381772,2020-12-25,1163,6.57,2.86,18.35,NaN,NaN
381783,2020-12-26,1163,6.57,2.86,18.35,NaN,NaN
381794,2020-12-27,1163,6.57,2.86,18.35,NaN,NaN
381805,2020-12-28,1163,6.57,2.86,18.35,NaN,NaN


In [125]:
result.to_csv('case1.csv')

In [127]:
CCs = pd.read_excel('data/CCs.xlsx')

In [128]:
CCs

,CCName,Lat,Lon
0,CC1,30.581417,-96.273549
1,CC2,29.941640,-95.621466
2,CC3,29.572316,-98.664508
3,CC4,33.014365,-96.914067
4,CC5,32.906213,-97.245081
5,CC6,30.317991,-95.546260
6,CC7,30.400980,-97.608475
7,CC8,30.256568,-97.866515
8,CC9,33.494254,-94.103031
9,CC10,29.987645,-94.006402


In [154]:
distance = np.array([(x,y) for x in SNFs['PID'] for y in CCs['CCName']])
distance = pd.DataFrame(distance, columns=['PID','CCName'])
distance[['PID']] = distance[['PID']].astype(int)

In [155]:
distance = distance.merge(CCs, on='CCName', how='left')
distance = distance.merge(PID_latlon, on='PID', how='left')

In [158]:
distance = distance.dropna(axis=0,how='any')

In [159]:
distance

,PID,CCName,Lat,Lon,latitude,longitude
19,2,CC1,30.581417,-96.273549,32.481913,-99.746158
20,2,CC2,29.941640,-95.621466,32.481913,-99.746158
21,2,CC3,29.572316,-98.664508,32.481913,-99.746158
22,2,CC4,33.014365,-96.914067,32.481913,-99.746158
23,2,CC5,32.906213,-97.245081,32.481913,-99.746158
...,...,...,...,...,...,...
34442,1813,CC15,29.519390,-95.426823,28.977992,-97.512121
34443,1813,CC16,32.627434,-96.855873,28.977992,-97.512121
34444,1813,CC17,31.612382,-94.645918,28.977992,-97.512121
34445,1813,CC18,32.917913,-97.202419,28.977992,-97.512121


In [162]:
from geopy.distance import great_circle

distance_list = []
for index, row in distance.iterrows():
    CC_location = (row['Lat'], row['Lon'])
    SNF_location = (row['latitude'], row['longitude'])
    distance_list.append(great_circle(CC_location, SNF_location).miles)
    
    print("progress: {0}%".format(round((index + 1) * 100 / len(distance), 2)), end="\r")

progress: 3.02%

In [165]:
distance['distance'] = np.array(distance_list)

/Users/waittim/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [166]:
distance

,PID,CCName,Lat,Lon,latitude,longitude,distance
19,2,CC1,30.581417,-96.273549,32.481913,-99.746158,243.011926
20,2,CC2,29.941640,-95.621466,32.481913,-99.746158,300.312068
21,2,CC3,29.572316,-98.664508,32.481913,-99.746158,210.983835
22,2,CC4,33.014365,-96.914067,32.481913,-99.746158,168.632633
23,2,CC5,32.906213,-97.245081,32.481913,-99.746158,148.350832
...,...,...,...,...,...,...,...
34442,1813,CC15,29.519390,-95.426823,28.977992,-97.512121,131.156102
34443,1813,CC16,32.627434,-96.855873,28.977992,-97.512121,255.140317
34444,1813,CC17,31.612382,-94.645918,28.977992,-97.512121,249.713993
34445,1813,CC18,32.917913,-97.202419,28.977992,-97.512121,272.840025


In [167]:
distance.to_csv('distance.csv')

In [2]:
distance = pd.read_csv("distance.csv")

In [3]:
distance.describe()

,Unnamed: 0,PID,Lat,Lon,latitude,longitude,distance
count,29203.000000,29203.000000,29203.000000,29203.000000,29203.000000,29203.000000,29203.000000
mean,17351.141835,913.744307,31.484370,-97.009193,31.305816,-97.570857,220.993879
std,9926.400673,522.442061,1.406797,2.552156,1.937754,2.343602,145.808204
min,19.000000,2.000000,29.519390,-106.433022,25.904869,-106.567020,0.174494
25%,8877.500000,468.000000,30.165193,-97.608475,29.746523,-98.518632,124.086919
50%,17318.000000,912.000000,31.612382,-96.914067,31.547724,-97.199425,198.712470
75%,25986.500000,1368.000000,32.917913,-95.546260,32.801143,-95.764466,278.544432
max,34446.000000,1813.000000,33.494254,-94.006402,36.460186,-93.732055,762.396621


In [4]:
distance.Lat.unique()

array([30.58141744, 29.94163954, 29.57231624, 33.01436534, 32.90621254,
       30.31799144, 30.40097954, 30.25656794, 33.49425424, 29.98764484,
       32.66487924, 30.16519284, 33.06050084, 33.21484704, 29.51938964,
       32.62743384, 31.61238244, 32.91791324, 31.94709894])

## Historical Data Estimation and Modeling 

In [3]:
main_df = pd.read_csv("case1.csv")

In [4]:
main_df

,Unnamed: 0,Date,PID,WorkLoadMode,WorkLoadMin,WorkLoadMax,latitude,longitude
0,0,2019-01-01,6,304.37,113.63,553.63,30.583101,-96.275736
1,1,2019-01-01,8,536.06,202.60,978.27,33.040040,-96.832425
2,2,2019-01-01,9,155.32,75.30,304.99,32.887383,-96.765846
3,3,2019-01-01,10,55.34,20.66,100.66,33.853630,-98.536196
4,4,2019-01-01,11,110.68,41.32,201.32,32.875667,-96.613629
...,...,...,...,...,...,...,...,...
382240,382240,2021-04-14,716,6.57,2.86,18.35,30.702646,-95.541119
382241,382241,2021-04-15,716,6.57,2.86,18.35,30.702646,-95.541119
382242,382242,2021-04-16,716,6.57,2.86,18.35,30.702646,-95.541119
382243,382243,2021-04-17,716,6.57,2.86,18.35,30.702646,-95.541119


In [5]:
main_df.describe()

,Unnamed: 0,PID,WorkLoadMode,WorkLoadMin,WorkLoadMax,latitude,longitude
count,382245.000000,382245.000000,382245.000000,382245.000000,382245.000000,325435.000000,325435.000000
mean,191122.000000,949.361043,37.538789,14.421279,91.378575,31.137876,-97.263887
std,110344.771161,532.230869,51.625680,19.533590,112.738147,1.977745,2.315481
min,0.000000,6.000000,6.570000,2.860000,18.350000,25.921078,-106.567020
25%,95561.000000,494.000000,13.140000,4.710000,33.930000,29.672370,-98.146819
50%,191122.000000,982.000000,19.710000,8.580000,55.050000,31.593978,-96.994318
75%,286683.000000,1421.000000,42.870000,17.160000,110.100000,32.825556,-95.562756
max,382244.000000,1813.000000,1203.520000,470.260000,2304.270000,36.370131,-93.786387


In [6]:
columns = []
count = 0
for i in ['mode','min','max']:
    main_df['cc_num_'+i] = np.ceil(main_df.iloc[:,count+3]/480)
    count += 1
    columns.append('cc_num_'+i)

In [28]:
main_df

,Date,PID,WorkLoadMode,WorkLoadMin,WorkLoadMax,latitude,longitude,cc_num_mode,cc_num_min,cc_num_max
0,2019-01-01,6,304.37,113.63,553.63,30.583101,-96.275736,1.0,1.0,2.0
1,2019-01-01,8,536.06,202.60,978.27,33.040040,-96.832425,2.0,1.0,3.0
2,2019-01-01,9,155.32,75.30,304.99,32.887383,-96.765846,1.0,1.0,1.0
3,2019-01-01,10,55.34,20.66,100.66,33.853630,-98.536196,1.0,1.0,1.0
4,2019-01-01,11,110.68,41.32,201.32,32.875667,-96.613629,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...
382240,2021-04-14,716,6.57,2.86,18.35,30.702646,-95.541119,1.0,1.0,1.0
382241,2021-04-15,716,6.57,2.86,18.35,30.702646,-95.541119,1.0,1.0,1.0
382242,2021-04-16,716,6.57,2.86,18.35,30.702646,-95.541119,1.0,1.0,1.0
382243,2021-04-17,716,6.57,2.86,18.35,30.702646,-95.541119,1.0,1.0,1.0


In [7]:
main_df.drop(columns = 'Unnamed: 0', inplace=True)
main_df.to_csv('new_case.csv',index=False)

In [8]:
columns.append('PID')
df1 = main_df[columns].groupby('PID').max()

In [9]:
df1.to_csv('history_cc_num.csv',index=False)

In [10]:
demo_snf = main_df.copy()
demo_snf['Date'] = pd.to_datetime(demo_snf['Date'])
demo_snf['Year'] = demo_snf['Date'].dt.year
demo_snf['Month'] = demo_snf['Date'].dt.month
demo_snf['Day'] = demo_snf['Date'].dt.day

In [11]:
# demo_snf.reset_index(drop=True, inplace=True)

In [12]:
demo_snf = demo_snf[demo_snf['Date'] < '2020-01-01']

In [13]:
predictors = ['PID','latitude','longitude','Year','Month','Day']
data_X = demo_snf.loc[:,predictors]
data_y = demo_snf.loc[:,'WorkLoadMode']

In [14]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

In [15]:
model = xgb.XGBRegressor()
model.fit(data_X,data_y)

XGBRegressor(base_score=0.5, booster=None, colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints=None,
             learning_rate=0.300000012, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints=None,
             n_estimators=100, n_jobs=0, num_parallel_tree=1,
             objective='reg:squarederror', random_state=0, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method=None,
             validate_parameters=False, verbosity=None)

In [16]:
df1=demo_snf[['PID','latitude','longitude']].drop_duplicates().reset_index(drop=True)
rownum = len(df1)

In [17]:
date = input('Enter date like 2020-10-01: ')

Enter date like 2020-10-01: 2020-10-08


In [18]:
y = int(date[:4])
m = int(date[5:7])
d = int(date[8:10])

In [19]:
Year = [y]*rownum
Month = [m]*rownum
Day = [d]*rownum
df1['Year'] = Year
df1['Month'] = Month
df1['Day'] = Day

In [20]:
test_x = df1.copy()

In [21]:
preds = model.predict(test_x)

In [22]:
preds = preds.tolist()
for i in range(len(preds)):
    if preds[i] <= 0:
        preds[i] = 0
number = [np.ceil(i/480) for i in preds]

In [23]:
output = pd.DataFrame({'PID':test_x['PID'].tolist(),'CC_preds':number,'Date':date})

In [24]:
output

,PID,CC_preds,Date
0,6,1.0,2020-10-08
1,8,1.0,2020-10-08
2,9,1.0,2020-10-08
3,10,0.0,2020-10-08
4,11,1.0,2020-10-08
...,...,...,...
1243,992,1.0,2020-10-08
1244,1011,1.0,2020-10-08
1245,1328,1.0,2020-10-08
1246,1516,1.0,2020-10-08


In [25]:
pd.DataFrame({'Date':date,'Lower Bound':np.ceil(sum(preds)/480)}, index=[0])

,Date,Lower Bound
0,2020-10-08,99.0


In [26]:
sum(preds)/480

98.7293819958965